In [ ]:
# 4. Softmax classification

# Import library
import numpy as np # 수치 연산을 위한 라이브러리
import tensorflow as tf # 머신러닝 프레임워크
import pandas as pd # 데이터 분석을 위한 라이브러리
from sklearn.preprocessing import OneHotEncoder # 카테고리 데이터를 one-hot encoding 하기 위한 라이브러리
from sklearn.model_selection import train_test_split

# data load
df = pd.read_csv('triangular.csv') #triangular.csv' 파일을 읽어서 데이터프레임 df에 저장


# one-hot encoding
from sklearn.preprocessing import OneHotEncoder #OneHotEncoder: 객체를 생성하여 클래스 열을 one-hot encoding 함
encoder = OneHotEncoder(sparse_output=False) #sparse를 False로 설정하면 넘파이 배열을 반환하고 True로 설정하면 희소 행렬로 반환한다
encoded_data = encoder.fit_transform(df[['class']]) #fit_transform 메서드를 사용하여 클래스 데이터에 맞게 인코더를 학습하고, one-hot encoding 결과를 encoded_data에 저장
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out()) #one-hot encoding 결과를 새로운 데이터프레임 encoded_df에 저장.
df = pd.concat([df, encoded_df], axis=1) #원본 데이터프레임 df에 one-hot encoding된 클래스 열을 추가하기 위해 concat 메서드를 사용


# 10개의 data만 선택
x_data = np.array(df.iloc[1:11,0:3], dtype=np.float32) #데이터프레임에서 1번째부터 10번째 행까지 처음 3열의 데이터를 선택하여 x_data에 NumPy 배열로 저장(특징 데이터).
y_data = np.array(df.iloc[1:11,4:], dtype=np.float32) #데이터프레임에서 1번째부터 10번째 행까지 4번째 열 이후의 모든 열 (one-hot encoding된 클래스 레이블)을 선택하여 y_data에 NumPy 배열로 저장(레이블 데이터).
print("x_data=", x_data)
print("y_data=", y_data)

############## Multinomial regression model
# 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=42)


# num_classes 설정 (예시)
num_classes = y_data.shape[1]  # y_data의 열 개수를 클래스 개수로 설정


# 모델 생성
model = tf.keras.Sequential([ #tf.keras.Sequential: 층을 순차적으로 쌓아 모델을 생성하는 간단한 모델
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)), # Dense() 첫 번째 은닉층을 정의
#128: 뉴런의 개수. 뉴런이 많을수록 더 복잡한 패턴을 학습할 수 있음
#activation='relu': 활성화 함수로 ReLU 함수를 사용. ReLU 함수는 음수 입력에 대해 0을 출력하고, 양수 입력에 대해서는 입력값을 그대로 출력.
#input_shape=(X_train.shape[1],): 입력 데이터의 형태를 지정. X_train.shape[1]은 입력 데이터의 특징 개수를 의미.
    tf.keras.layers.Dense(64, activation='relu'), #Dense() 두 번째 은닉층을 정의
    tf.keras.layers.Dense(num_classes, activation='softmax') #Dense(num_classes, activation='softmax'): 출력층을 정의
])
# num_classes는 클래스의 개수를 의미하며, softmax 활성화 함수를 사용하여 각 클래스에 대한 확률을 출력




# 모델 컴파일 model.compile: 모델을 학습하기 위한 설정
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), #옵티마이저로 Adam 옵티마이저를 사용하고, 학습률을 0.001로 설정 / Adam 옵티마이저는 일반적으로 빠르게 수렴하는 것으로 알려져 있음
              loss='categorical_crossentropy', #손실 함수로 categorical crossentropy를 사용 / 다중 클래스 분류 문제에서 일반적으로 사용되는 손실 함수
              metrics=['accuracy']) #모델 평가 시 정확도를 측정


# 모델 학습    model.fit: 모델을 학습시

model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))
#print("model.fit=", model.fit)
  #X_train, y_train: 학습 데이터와 레이블
  #epochs=50: 전체 데이터를 50번 반복하여 학습
  #batch_size=32: 한 번에 32개의 데이터를 사용하여 학습
  #validation_data=(X_test, y_test): 검증 데이터를 지정하여 학습 중에 모델의 성능을 평가



# Model parameter
# 모델 파라미터를 작성하세요. / 가중치 행렬 W과 편향 벡터 b을 텐서플로우 변수 (Variable)로 정의
#tf.random.normal 함수를 사용하여 임의의 값으로 초기화
W = tf.Variable(tf.random.normal([3, 3])) #[3, 3]: 가중치 행렬의 크기는 입력 특징의 수 (3)와 클래스의 수 (3)를 반영
b = tf.Variable(tf.random.normal([3])) #[3]: 편향 벡터의 크기는 클래스의 수와 동일
#print("w=", W)
#print("b=", b)
# learning rate
learning_rate = 0.01 #학습률

# Softmax classifier
def softmax_classifier(): # 소프트맥스 분류기를 정의하는 함수
    with tf.GradientTape() as tape: # 그래디언트 계산을 위한 컨텍스트를 생성
        # 변경된 코드
        logits = tf.matmul(x_data, W) + b # 입력 데이터 x_data와 가중치 행렬 W의 행렬곱에 편향 벡터 b를 더하여 로짓 값을 계산
        softmax_output = tf.nn.softmax(logits) # 로짓 값을 소프트맥스 함수에 적용하여 각 클래스에 대한 확률을 계산

        # Cross-entropy (라이브러리 사용 X)
        cost = -tf.reduce_mean(tf.reduce_sum(y_data * tf.math.log(softmax_output), axis=1)) # 교차 엔트로피 손실을 계산
        # (y_data * tf.math.log(softmax_output): 각 클래스에 대한 실제 레이블과 예측 확률의 로그의 곱을 계산
        # tf.reduce_sum(..., axis=1): 각 샘플에 대해 클래스별 손실을 합산
        # tf.reduce_mean(...): 전체 샘플에 대한 평균 손실을 계산



        # Gradient 계산
        gradients = tape.gradient(cost, [W, b]) #손실 함수에 대한 가중치와 편향의 그래디언트를 계산
        #print("gradients=", gradients)
        # Weight update
        tf.optimizers.SGD(learning_rate).apply_gradients(zip(gradients, [W, b])) #경사하강법 옵티마이저를 사용하여 가중치와 편향을 업데이트

        # 손실 값 반환
        return cost

############### Learning
for step in range(5001): #5001번의 학습 반복을 수행
    cost = softmax_classifier() #소프트맥스 분류기 함수를 호출하여 손실 값을 계산하고 반환

    if step % 500 == 0: #500번마다 학습 진행 상황을 출력
        # Softmax 분류기를 작성하세요.
        model_LC = tf.matmul(x_data,W) + b #모델의 출력을 계산
        model = tf.argmax(tf.nn.softmax(model_LC),1) #출력값을 클래스 인덱스로 변환

        # accuracy 계산 /  accuracy, cost 출력
        accuracy = tf.reduce_mean(tf.cast(tf.equal(model, tf.argmax(y_data,1)),tf.float32)) #정확도를 계산
        print(step, 'Accuracy : ', accuracy.numpy(), 'Cost : ', cost.numpy()) #학습 단계, 정확도, 손실 값을 출력


x_data= [[9.46 8.48 7.74]
 [7.23 8.58 8.4 ]
 [5.73 3.82 9.95]
 [3.07 6.1  6.83]
 [6.49 4.28 9.98]
 [4.11 9.14 3.26]
 [8.69 7.11 8.81]
 [8.85 9.06 9.75]
 [9.73 7.29 9.59]
 [8.67 9.21 7.82]]
y_data= [[1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.1250 - loss: 1.6629 - val_accuracy: 0.0000e+00 - val_loss: 1.9203
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step - accuracy: 0.1250 - loss: 1.3065 - val_accuracy: 0.0000e+00 - val_loss: 1.3948
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.5000 - loss: 1.0252 - val_accuracy: 0.5000 - val_loss: 0.9660
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.8750 - loss: 0.8234 - val_accuracy: 1.0000 - val_loss: 0.6348
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.7500 - loss: 0.7002 - val_accuracy: 1.0000 - val_loss: 0.4140
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - accuracy: 0.7500 - loss: 0.6434 - val_accuracy: 1.0000 - val_loss: 0.2837
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step - accuracy: 0.7500 - loss: 0.6274 - val_accuracy: 1.0000 - val_loss: 0.2155
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.7500 - loss: 0.6247 - val_accuracy: 1.0000 - val_loss: